In [ ]:
# Decide whether or not running on Kaggle
import os
KAGGLE_RUN = True
N_TRAIN_ROWS = 10 * 1000000
if os.path.isdir('/home/ubuntu'):
    KAGGLE_RUN = False
    
if KAGGLE_RUN:
    model_name = 'model8/model8.txt'
else:
    model_name = 'model8.txt'
    
# Libraries
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score

from collections import defaultdict
import lightgbm as lgb
import random
if not KAGGLE_RUN:
    import sys
    sys.path.insert(0, './input')
    import local_work
import riiideducation

# Boto3 Setup and Download Files
if not KAGGLE_RUN:
    local_work = local_work.local_work()
    local_work.download_riiid_files()
    
# Column definitions    
train_cols = ['user_id', 'timestamp', 'task_container_id'
              ,'content_id', 'content_type_id', 'answered_correctly'
             ,'prior_question_elapsed_time', 'prior_question_had_explanation']
target = 'answered_correctly'

In [ ]:
# Load data
if not KAGGLE_RUN:
    df_train = local_work.get_train_data(train_cols, nrow=N_TRAIN_ROWS)
    df_questions = local_work.get_questions_data()
else:
    import datatable as dt
    if N_TRAIN_ROWS is None:
        df_train = dt.fread('../input/riiid-test-answer-prediction/train.csv'
                            ,columns=set(train_cols)).to_pandas()        
    else:
        df_train = dt.fread('../input/riiid-test-answer-prediction/train.csv'
                            ,columns=set(train_cols)
                            ,max_nrows=N_TRAIN_ROWS).to_pandas()
    df_questions = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv'
                                ,usecols=[0, 3]
                                ,dtype={'question_id': 'int16', 'part': 'int8'}
                               )
    

# Cleaning function
def clean(df, target, df_questions, is_train):
    # Only keep question rows
    df = df[df['content_type_id'] == 0].reset_index(drop=True)
    
    # Assume False if NA (Only for first question ? )
    df['prior_question_had_explanation'].fillna(False, inplace=True)  
    
    # Alter columns types
    tmp = {'user_id': 'int32'
            ,'content_id': 'int16'
            ,target: 'int8'
            ,'prior_question_elapsed_time': 'float32' 
            ,'prior_question_had_explanation': 'bool'
            }
    if not is_train:
        del tmp[target]
    df = df.astype(tmp)
    
    # Merge questions
    df = pd.merge(df, df_questions, left_on='content_id', right_on='question_id', how='left')
        
    return(df)

# Feature Engineering function
def do_fe(df
          ,is_train
          ,content_agg=None
          ,user_sum_dict=None
          ,user_count_dict=None
          ,content_sum_dict=None
          ,content_count_dict=None
          ,user_prev_task_sum_dict=None
          ,user_prev_task_count_dict=None
          ,user_prev_task_timestamp_dict=None
         ):

    if is_train:
        # Get sum/count of answers in current, previous, and cumulative tasks.
        df_tasks = df\
                    .groupby(['user_id', 'task_container_id'], sort=False)[target]\
                    .agg(['sum', 'count'])\
                    .rename(columns={'sum': 'task_sum', 'count': 'task_count'})
        df_tasks['prev_task_sum'] = df_tasks.groupby('user_id')['task_sum'].shift()
        df_tasks['prev_task_count'] = df_tasks.groupby('user_id')['task_count'].shift()
        df_tasks['prev_task_cum_sum'] = df_tasks.groupby('user_id')['prev_task_sum'].cumsum()
        df_tasks['prev_task_cum_count'] = df_tasks.groupby('user_id')['prev_task_count'].cumsum()
        
        # FE accuracy on previous task(s)
        df_tasks['prev_task_correctness'] = df_tasks['prev_task_sum']/df_tasks['prev_task_count']
        df_tasks['prev_task_cum_correctness'] = df_tasks['prev_task_cum_sum']/\
                                                    df_tasks['prev_task_cum_count']

        # Drop undesired columns
        df_tasks.drop(columns=['prev_task_sum', 'prev_task_count', 'prev_task_cum_sum'], inplace=True)
        
        # Join df_tasks to df_train
        df = df.merge(df_tasks, on=['user_id', 'task_container_id'])

        # Garbage College
        del df_tasks
        gc.collect()
        
        # Add time since last task
        df_tasks = df_train\
                    .groupby(['user_id', 'task_container_id'], sort=False)[['timestamp']]\
                    .mean()
        df_tasks['time_since_prev_task_1'] = df_tasks['timestamp'] - df_tasks.groupby('user_id')['timestamp'].shift()
        #df_tasks['time_since_prev_task_2'] = df_tasks['timestamp'] - df_tasks.groupby('user_id')['timestamp'].shift(2)
        
        # Drop undesired columns
        df_tasks.drop(columns=['timestamp'], inplace=True)
        
        # Join df_tasks to df_train
        df = df.merge(df_tasks, on=['user_id', 'task_container_id'])
    
    else:
        # Initialize user/content sums
        user_sum = np.zeros(len(df), dtype=np.int16)
        user_count = np.zeros(len(df), dtype=np.int16)
        user_prev_sum = np.zeros(len(df), dtype=np.int16)
        user_prev_count = np.zeros(len(df), dtype=np.int16)
        user_prev_timestamp = np.zeros(len(df), dtype=np.int64)

        # Get user historical info
        for i, user_id in enumerate(df['user_id'].values):
            user_sum[i] = user_sum_dict[user_id]
            user_count[i] = user_count_dict[user_id]
            user_prev_sum[i] = user_prev_task_sum_dict[user_id]
            user_prev_count[i] = user_prev_task_count_dict[user_id]
            user_prev_timestamp[i] = user_prev_task_timestamp_dict[user_id]
            
        # Add columns to df_test
        with np.errstate(divide='ignore', invalid='ignore'):
            df['prev_task_cum_correctness'] = user_sum / user_count
            df['prev_task_correctness'] = user_prev_sum / user_prev_count
        df['prev_task_cum_count'] = user_count
        df['time_since_prev_task_1'] = df['timestamp'] - user_prev_timestamp
        
        # Add task count
        df['task_count'] = df.groupby('user_id')['user_id'].transform('count')
        
    # Content Columns - Not updating throughout testing phase
    df['content_count'] = df['content_id'].map(content_agg['count']).astype('int32')
    df['content_avg_correctness'] = df['content_id'].map(content_agg['sum'] / content_agg['count'])
    
    # Timestamp divided by number of tasks
    df['timestamp_div_task_cum_count'] = df['timestamp'] / df['prev_task_cum_count']
    
    return(df)

In [ ]:
# Clean
df_train = clean(df_train
                 ,target
                 ,df_questions
                 ,is_train=True
                )

# Capture user and content information
content_agg = df_train.groupby('content_id')[target].agg(['sum', 'count'])
user_agg = df_train.groupby('user_id')[target].agg(['sum', 'count'])

# Get information on the last user task
df_last_row = df_train[['user_id', 'task_container_id', 'timestamp']].groupby('user_id').tail(1)
df_last_task = df_train.merge(df_last_row[['user_id', 'task_container_id']]
                              , on=['user_id', 'task_container_id']
                              , how='inner')

if KAGGLE_RUN:
    del df_train
    gc.collect()   

# This is for testing set
user_prev_task = df_last_task\
            .groupby(['user_id', 'task_container_id'])[target]\
            .agg(['sum', 'count'])\
            .rename(columns={'sum': 'prev_task_sum', 'count': 'prev_task_count'})\
            .reset_index(level='task_container_id', drop=True)
    
# Feature engineer
if not KAGGLE_RUN:
    print('Starting FE')
    df_train = do_fe(df_train, is_train=True, content_agg = content_agg)
    
    # Delete uneeded columns
    df_train.drop(columns=['task_sum'
                           ,'question_id'
                           ,'content_id'
                           ,'task_container_id'
                           ,'content_type_id'
                          ]
                  ,inplace=True)
    gc.collect()
    print('Finished FE')
    

Model

In [ ]:
# Define features
features = [
            # Task related
            'prev_task_correctness'
            ,'prev_task_cum_count'
            ,'prev_task_cum_correctness'
            ,'task_count'
            ,'time_since_prev_task_1'
            # Content related
            ,'content_avg_correctness'
            ,'content_count'
            # From df_train
            ,'timestamp_div_task_cum_count'
            ,'timestamp'
            ,'prior_question_elapsed_time'
            ,'prior_question_had_explanation'
            # From df_questions
            ,'part'
            ]

if not KAGGLE_RUN:
    pass
    #assert(len(features) == (df_train.shape[1] - 2)) # Minuse 2 for target and user_id
    
if not KAGGLE_RUN:    
    
    # - # - # Make Validation Set # - # - #
    # Add entire user histories to validation set
    tmp = df_train.user_id.unique()
    val_full_ids = np.random.choice(tmp, int(.05 * tmp.shape[0]))
    df_valid_full = df_train.loc[df_train['user_id'].isin(val_full_ids)]
    df_train.drop(df_valid_full.index, inplace=True)

    # Add start of user histories to validation set
    tmp = df_train.user_id.unique()
    val_start_ids = np.random.choice(tmp, int(.07 * tmp.shape[0]))
    df_valid_start = df_train.loc[df_train['user_id'].isin(val_start_ids)].groupby('user_id').head(100)
    df_train.drop(df_valid_start.index, inplace=True)

    # Add ending of user histories to validation set
    tmp = df_train.user_id.unique()
    val_end_ids = np.random.choice(tmp, int(.07 * tmp.shape[0]))
    df_valid_end = df_train.loc[df_train['user_id'].isin(val_end_ids)].groupby('user_id').tail(100)
    
    # Clean up
    df_train.drop(df_valid_end.index, inplace=True)
    del tmp
    del val_full_ids
    del val_start_ids
    del val_end_ids
    gc.collect()
    
    # Combine the sets
    print('Joining validation sets')
    df_valid = df_valid_full.append([df_valid_start, df_valid_end])
    
    # - # - # Build and Save Model # - # - #
    if True:
        print('Building Model')
        model = local_work.make_model(df_train, df_valid, target, features, learning_rate=1)
        model.save_model(f'./output/' + model_name)
        lgb.plot_importance(model, importance_type='gain')
    else:
        model = lgb.Booster(model_file='./output/' + model_name)
    
else:
    model = lgb.Booster(model_file='/kaggle/input/' + model_name)

In [ ]:
# Setup dicts
user_sum_dict = user_agg['sum'].astype('int16').to_dict(defaultdict(int))
user_count_dict = user_agg['count'].astype('int16').to_dict(defaultdict(int))
user_prev_task_sum_dict = user_prev_task['prev_task_sum']\
                            .astype('int16').to_dict(defaultdict(int))
user_prev_task_count_dict = user_prev_task['prev_task_count']\
                            .astype('int16').to_dict(defaultdict(int))
df_tmp = df_last_row.drop(columns=['task_container_id']).set_index('user_id')
user_prev_task_timestamp_dict = df_tmp['timestamp']\
                                .astype('int64').to_dict(defaultdict(int))

# Make env
env = riiideducation.make_env()
iter_test = env.iter_test()
df_prior_test = None

# Make predictions
for (df_test, df_sample_prediction) in iter_test:
    
    if df_prior_test is not None:
        # Attach whether or not users answered correctly
        df_prior_test[target] = eval(df_test['prior_group_answers_correct'].iloc[0])
        df_prior_test = df_prior_test[df_prior_test[target] != -1].reset_index(drop=True)

        user_ids = df_prior_test['user_id'].values
        content_ids = df_prior_test['content_id'].values
        targets = df_prior_test[target].values
        
        # If user present in current batch, reset the previous task dictionary
        for user_id in df_test['user_id'].unique():
            user_prev_task_sum_dict[user_id] = 0
            user_prev_task_count_dict[user_id] = 0

        # Update user and content dictionaries
        for user_id, content_id, timestamp, answered_correctly in zip(df_prior_test['user_id'].values
                                                                       ,df_prior_test['content_id'].values
                                                                       ,df_prior_test['timestamp'].values
                                                                       ,df_prior_test[target].values):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            user_prev_task_sum_dict[user_id] += answered_correctly
            user_prev_task_count_dict[user_id] += 1
            user_prev_task_timestamp_dict[user_id] = timestamp

    # Make a copy of df_test for the next iteration
    df_prior_test = df_test.copy()
 
    # Clean
    df_test = clean(df_test
                     ,target
                     ,df_questions
                     ,is_train=False
                    )
    
    # Feature engineer
    df_test = do_fe(df_test
                    ,is_train=False
                    ,content_agg = content_agg
                    ,user_sum_dict = user_sum_dict
                    ,user_count_dict = user_count_dict
                    ,user_prev_task_sum_dict = user_prev_task_sum_dict
                    ,user_prev_task_count_dict = user_prev_task_count_dict
                    ,user_prev_task_timestamp_dict = user_prev_task_timestamp_dict
                    )
    
    # Make prediction
    df_test[target] = model.predict(df_test[features])
    env.predict(df_test[['row_id', target]])
    
print("Finished")

Explorations

In [ ]:
if False:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()

    all_dfs = []
    for (df_test, df_sample_prediction) in iter_test:
        all_dfs.append(df_test)
        df_test[target] = .5
        env.predict(df_test[['row_id', target]])

    for i in range(3):
        print(all_dfs[i].shape[0])
        print(len(all_dfs[i].user_id.unique()) - all_dfs[i].shape[0])

In [ ]:
all_dfs[1]

In [ ]:
df_test